In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.model_selection import LeaveOneOut
seed=911
###########import packages##########
import catboost
import xgboost
import lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import *
import pickle
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import  *
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
# from keras.wrappers import scikit_learn
from scikeras.wrappers import KerasClassifier, KerasRegressor
###########loading data##########
loo = LeaveOneOut()
# %matplotlib



In [ ]:
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
def gridsearch(model,param,algorithm_name,X_train,y_train,X_test,y_test):
    grid = GridSearchCV(model,param_grid=param,scoring='neg_mean_absolute_error',cv=10,n_jobs=8,verbose=-1)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    ####Train####
    print("=========Train===========")
    prediction_train = best_model.predict(X_train)
    real_train=y_train.values
    prediction_train_series=pd.Series(prediction_train)
    real_train_series=pd.Series(real_train)
    corr_ann_train = round(prediction_train_series.corr(real_train_series), 5)
    error_val_train= compute_mae_mse_rmse(prediction_train,real_train)
    print(error_val_train)
    ####test####
    print("=========Test===========")
    prediction_test = best_model.predict(X_test)
    real_test=y_test.values
    prediction_test_series=pd.Series(prediction_test)
    real_test_series=pd.Series(real_test)
    corr_ann_test = round(prediction_test_series.corr(real_test_series), 5)
    error_val_test= compute_mae_mse_rmse(prediction_test,real_test)
    print(error_val_test)
    ####All####
    print("=========All===========")
    prediction_all = best_model.predict(data_input_full)
    real_all=data_output_full.values
    prediction_all_series=pd.Series(prediction_all)
    real_all_series=pd.Series(real_all)
    corr_ann_all = round(prediction_all_series.corr(real_all_series), 5)
    error_val_all= compute_mae_mse_rmse(prediction_all,real_all)
    print(error_val_all)
    
    print(algorithm_name)
    best_score=grid.best_score_
    print('Best Regressor:',grid.best_params_,'Best Score:', best_score)
    print('R2 TEST',error_val_test[3])
    fig=plt.figure()
    ax = fig.add_subplot(111)
    x_y_x=np.arange(-4,4,0.01)
    x_y_y=np.arange(-4,4,0.01)
    ax.scatter(prediction_train,real_train,c='blue',label='Train',alpha=0.25)
    ax.scatter(prediction_test,real_test,c='red',label='Test',alpha=0.75)
    ax.plot(x_y_x,x_y_y,c='black')
    plt.legend()
    plt.xlabel('Predicted_Overpotential@10 mA cm-2')
    plt.ylabel('Real_Overpotential@10 mA cm-2')
    return best_model,error_val_test[3]

In [ ]:
fl = open(r'./database_full_st.pkl','rb')
database_full=pickle.load(fl)

In [ ]:
data_input_full=database_full.iloc[:,0:55]
data_output_full=database_full.iloc[:,55]
X_train,X_test,y_train,y_test=train_test_split(data_input_full,data_output_full,test_size=0.1,random_state=seed)

In [ ]:
model_XGRegressor=XGBRegressor(random_state=1)
param_xg={
'booster':['gbtree'],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[100,200,400],
'max_depth':[3,5,7,9,11,13,-1],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
# 'booster': ['gbtree'], 'learning_rate': [0.05], 'max_depth': [11], 'n_estimators': [100], 'reg_alpha': [1e-05], 'reg_lambda': [0], 'subsample': [0.45 ]   
}
XG_full,XG_full_score=gridsearch(model_XGRegressor,param_xg,'XGBoost',X_train,y_train,X_test,y_test)

In [ ]:
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[50,100,200,400],
'max_depth':[3,5,7,9,11,13,16],
'criterion':['friedman_mse','mae','mse'],
'max_features':['auto','sqrt','log2'],
'loss':['ls', 'lad', 'huber', 'quantile']
#     'criterion': ['friedman_mse'], 'learning_rate': [0.05], 'loss': ['lad'], 'max_depth': [9], 'max_features': ['auto'], 'n_estimators': [100]
}
GB_full,GB_full_score=gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost',X_train,y_train,X_test,y_test)

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'n_estimators':[50,100,200,400,None],
'max_depth':[3,5,7,9,11,None],
'criterion':['mse','mae'],
'max_features':['auto','sqrt','log2']
# 'criterion': ['mae'], 'max_depth': [11], 'max_features': ['auto'], 'n_estimators': [400]
}
RF_full,RF_full_score=gridsearch(model_RandomForestRegressor,param_RF,'Random Forest',X_train,y_train,X_test,y_test)

In [ ]:
import shap
ELE=data_input_full.iloc[:,0:36]
SYN=data_input_full.iloc[:,36:]

In [ ]:
###############################################################################################################################
#########################################################XGBoost###############################################################
###############################################################################################################################

################################
#############overall############
################################
explainer = shap.TreeExplainer(XG_full, data_input_full)
shap_values_XG = explainer(data_input_full,check_additivity=False)
#####summary plot#####
shap.summary_plot(shap_values_XG,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
XG_v = shap_values_XG.cohorts(2).abs.mean(0)
shap.plots.bar(XG_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(shap_values_XG, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_XG,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
#########################################################XGBoost###############################################################
##############################################################################################################################

################################
#############ELEMENT############
################################
shap_values_XG_ELE = shap_values_XG[:,0:36]
#####summary plot#####
shap.summary_plot(shap_values_XG_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
XG_v = shap_values_XG_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(XG_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(shap_values_XG_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_XG_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
#########################################################XGBoost###############################################################
##############################################################################################################################

################################
#############SYNTHESIS##########
################################
shap_values_XG_SYN = shap_values_XG[:,36:]
#####summary plot#####
shap.summary_plot(shap_values_XG_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
XG_v = shap_values_XG_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(XG_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(shap_values_XG_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_XG_SYN,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################Random Forest############################################################
###############################################################################################################################

################################
#############overall############
################################
explainer = shap.TreeExplainer(RF_full, data_input_full)
shap_values_RF = explainer(data_input_full,check_additivity=False)
#####summary plot#####
shap.summary_plot(shap_values_RF,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
RF_v = shap_values_RF.cohorts(2).abs.mean(0)
shap.plots.bar(RF_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(shap_values_RF, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_RF,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################Random Forest############################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
shap_values_RF_ELE = shap_values_RF[:,0:36]
#####summary plot#####
shap.summary_plot(shap_values_RF_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
RF_v = shap_values_RF_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(RF_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(shap_values_RF_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_RF_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
######################################################Random Forest############################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
shap_values_RF_SYN = shap_values_RF[:,36:]
#####summary plot#####
shap.summary_plot(shap_values_RF_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
RF_v = shap_values_RF_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(RF_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(shap_values_RF_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_RF_SYN,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
#####################################################Gradient Boost############################################################
###############################################################################################################################

################################
#############overall############
################################
explainer = shap.TreeExplainer(GB_full, data_input_full)
shap_values_GB = explainer(data_input_full,check_additivity=False)
#####summary plot#####
shap.summary_plot(shap_values_GB,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
GB_v = shap_values_GB.cohorts(2).abs.mean(0)
shap.plots.bar(GB_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(shap_values_GB, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_GB,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
#####################################################Gradient Boost############################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
shap_values_GB_ELE = shap_values_GB[:,0:36]
#####summary plot#####
shap.summary_plot(shap_values_GB_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
GB_v = shap_values_GB_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(GB_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(shap_values_GB_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_GB_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
#####################################################Gradient Boost############################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
shap_values_GB_SYN = shap_values_GB[:,36:]
#####summary plot#####
shap.summary_plot(shap_values_GB_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
GB_v = shap_values_GB_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(GB_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(shap_values_GB_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(shap_values_GB_SYN,max_display=25,plot_width=20)

In [ ]:
weighted_SHAP_ALL=(float(GB_full_score)*shap_values_GB+float(XG_full_score)*shap_values_XG+float(RF_full_score)*shap_values_RF)/(GB_full_score+XG_full_score+RF_full_score)
weighted_SHAP_ELE=(float(GB_full_score)*shap_values_GB_ELE+float(XG_full_score)*shap_values_XG_ELE+float(RF_full_score)*shap_values_RF_ELE)/(GB_full_score+XG_full_score+RF_full_score)
weighted_SHAP_SYN=(float(GB_full_score)*shap_values_GB_SYN+float(XG_full_score)*shap_values_XG_SYN+float(RF_full_score)*shap_values_RF_SYN)/(GB_full_score+XG_full_score+RF_full_score)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############overall############
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_ALL,feature_names=list(data_input_full.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
WE_v = weighted_SHAP_ALL.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(data_input_full, data_output_full) 
shap.plots.bar(weighted_SHAP_ALL, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(weighted_SHAP_ALL,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_ELE,feature_names=list(ELE.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
WE_v = weighted_SHAP_ELE.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(ELE, data_output_full) 
shap.plots.bar(weighted_SHAP_ELE, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(weighted_SHAP_ELE,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_SYN,feature_names=list(SYN.columns),max_display=25,alpha=0.5)
#####cohorts plot#####
WE_v = weighted_SHAP_SYN.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=18)
#####clustered plot#####
clustering = shap.utils.hclust(SYN, data_output_full) 
shap.plots.bar(weighted_SHAP_SYN, max_display=25,clustering=clustering,clustering_cutoff=0.5)
#####heat map#####
shap.plots.heatmap(weighted_SHAP_SYN,max_display=25,plot_width=20)

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############overall############
################################
#####cohorts plot#####
fig = plt.figure()
WE_v = weighted_SHAP_ALL.cohorts(2).abs.mean(0)
shap.plots.bar(WE_v,max_display=6,show=False)
plt.gcf().set_size_inches(4,7)
plt.show()

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############ELEMENT############
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_ELE,feature_names=list(ELE.columns),max_display=5,alpha=0.5,plot_size=(2,3))

In [ ]:
###############################################################################################################################
##################################################Weighted ENSEMBLE############################################################
###############################################################################################################################

################################
#############SYNTHESIS##########
################################
#####summary plot#####
shap.summary_plot(weighted_SHAP_SYN,feature_names=list(SYN.columns),max_display=5,alpha=0.5,plot_size=(2,3))

In [ ]:
#####overall weighted ranking####
shap.plots.bar(weighted_SHAP_ALL,max_display=25)

In [ ]:
shap.plots.bar(weighted_SHAP_ELE,max_display=25)

In [ ]:
shap.plots.bar(weighted_SHAP_SYN,max_display=25)

In [ ]:
data_input_full.columns

In [ ]:
from sklearn.inspection import partial_dependence
import matplotlib.pyplot as plt

def pdp_average_plot_2d(models, features, data_input):
    """
    Plot the averaged 2D partial dependence plot for a list of models.

    Parameters:
    - models: List of models to compute the PDP.
    - features: List of two features to compute the PDP.
    - data_input: Data input for computing the PDP.
    """

    # Initialize the average_pdp with zeros
    average_pdp = 0

    # Compute the partial dependence for each model and accumulate
    for model in models:
        pdp, axes = partial_dependence(model, data_input, features)
        average_pdp += pdp

    # Average the accumulated PDP results
    average_pdp /= len(models)

    # Plot the averaged results
    fig, ax = plt.subplots(figsize=(10, 8))
    contour = ax.contourf(axes[0], axes[1], average_pdp[0].T, alpha=0.75)
    ax.set_xlabel(features[0])
    ax.set_ylabel(features[1])
    fig.colorbar(contour)
    plt.show()

In [ ]:
shap.dependence_plot('Metal_Dopant_2 Atomic Radius', shap_values_XG.values, data_input_full,
                     interaction_index='Metal_Dopant_2 Number of d electrons',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Proportion in Precursor ', shap_values_XG.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Proportion in Precursor ',alpha=0.75)
shap.dependence_plot('Metal_Dopant_1 Number of d electrons', shap_values_XG.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Stability Test Time (h)', shap_values_XG.values, data_input_full,
                     interaction_index='Annealing Atomosphere Reductive (0/1)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (Celcius Degree)', shap_values_XG.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)
shap.dependence_plot('Hydrothermal Time (min)', shap_values_XG.values, data_input_full,xmax=250,
                     interaction_index='Support Material Loading (mg cm -2)',alpha=0.75)

In [ ]:
# List of models
models_list = [XG_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_2 Atomic Radius', 'Metal_Dopant_2 Number of d electrons'],
    ['Metal_Dopant_2 Proportion in Precursor ', 'Metal_Dopant_1 Proportion in Precursor '],
    ['Metal_Dopant_1 Number of d electrons', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Stability Test Time (h)', 'Annealing Atomosphere Reductive (0/1)'],
    ['Annealing Temperature (Celcius Degree)', 'Annealing Time (min)'],
    ['Hydrothermal Time (min)', 'Support Material Loading (mg cm -2)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)

In [ ]:
shap.dependence_plot('Metal_Dopant_2 Atomic Radius', shap_values_GB.values, data_input_full,
                     interaction_index='Metal_Dopant_2 Number of d electrons',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Proportion in Precursor ', shap_values_GB.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Proportion in Precursor ',alpha=0.75)
shap.dependence_plot('Metal_Dopant_1 Number of d electrons', shap_values_GB.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Stability Test Time (h)', shap_values_GB.values, data_input_full,
                     interaction_index='Annealing Atomosphere Reductive (0/1)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (Celcius Degree)', shap_values_GB.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)
shap.dependence_plot('Hydrothermal Time (min)', shap_values_GB.values, data_input_full,xmax=250,
                     interaction_index='Support Material Loading (mg cm -2)',alpha=0.75)

In [ ]:
# List of models
models_list = [GB_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_2 Atomic Radius', 'Metal_Dopant_2 Number of d electrons'],
    ['Metal_Dopant_2 Proportion in Precursor ', 'Metal_Dopant_1 Proportion in Precursor '],
    ['Metal_Dopant_1 Number of d electrons', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Stability Test Time (h)', 'Annealing Atomosphere Reductive (0/1)'],
    ['Annealing Temperature (Celcius Degree)', 'Annealing Time (min)'],
    ['Hydrothermal Time (min)', 'Support Material Loading (mg cm -2)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)

In [ ]:
shap.dependence_plot('Metal_Dopant_2 Atomic Radius', shap_values_RF.values, data_input_full,
                     interaction_index='Metal_Dopant_2 Number of d electrons',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Proportion in Precursor ', shap_values_RF.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Proportion in Precursor ',alpha=0.75)
shap.dependence_plot('Metal_Dopant_1 Number of d electrons', shap_values_RF.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Stability Test Time (h)', shap_values_RF.values, data_input_full,
                     interaction_index='Annealing Atomosphere Reductive (0/1)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (Celcius Degree)', shap_values_RF.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)
shap.dependence_plot('Hydrothermal Time (min)', shap_values_RF.values, data_input_full,xmax=250,
                     interaction_index='Support Material Loading (mg cm -2)',alpha=0.75)

In [ ]:
# List of models
models_list = [RF_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_2 Atomic Radius', 'Metal_Dopant_2 Number of d electrons'],
    ['Metal_Dopant_2 Proportion in Precursor ', 'Metal_Dopant_1 Proportion in Precursor '],
    ['Metal_Dopant_1 Number of d electrons', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Stability Test Time (h)', 'Annealing Atomosphere Reductive (0/1)'],
    ['Annealing Temperature (Celcius Degree)', 'Annealing Time (min)'],
    ['Hydrothermal Time (min)', 'Support Material Loading (mg cm -2)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)

In [ ]:
shap.dependence_plot('Metal_Dopant_2 Atomic Radius', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Metal_Dopant_2 Number of d electrons',alpha=0.75)
shap.dependence_plot('Metal_Dopant_2 Proportion in Precursor ', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Proportion in Precursor ',alpha=0.75)
shap.dependence_plot('Metal_Dopant_1 Number of d electrons', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Metal_Dopant_1 Relative Atomic Mass',alpha=0.75)
###############################################################################################################
shap.dependence_plot('Stability Test Time (h)', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Annealing Atomosphere Reductive (0/1)',alpha=0.75)
shap.dependence_plot('Annealing Temperature (Celcius Degree)', weighted_SHAP_ALL.values, data_input_full,
                     interaction_index='Annealing Time (min)',alpha=0.75)
shap.dependence_plot('Hydrothermal Time (min)', weighted_SHAP_ALL.values, data_input_full,xmax=250,
                     interaction_index='Support Material Loading (mg cm -2)',alpha=0.75)

In [ ]:
# List of models
models_list = [XG_full,GB_full,RF_full]

# Feature pairs and data input for the plots
feature_pairs = [
    ['Metal_Dopant_2 Atomic Radius', 'Metal_Dopant_2 Number of d electrons'],
    ['Metal_Dopant_2 Proportion in Precursor ', 'Metal_Dopant_1 Proportion in Precursor '],
    ['Metal_Dopant_1 Number of d electrons', 'Metal_Dopant_1 Relative Atomic Mass'],
    ['Stability Test Time (h)', 'Annealing Atomosphere Reductive (0/1)'],
    ['Annealing Temperature (Celcius Degree)', 'Annealing Time (min)'],
    ['Hydrothermal Time (min)', 'Support Material Loading (mg cm -2)']
]

# Plotting the PDP for each feature pair
for features in feature_pairs:
    pdp_average_plot_2d(models_list, features, data_input_full)